In [1]:
import pandas as pd
import datetime as dt

# Load data
df = pd.read_csv('../data/processed_retail.csv', parse_dates=['InvoiceDate'])

# Reference date (Today's date in dataset context)
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

# RFM Calculation
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, # Recency
    'Invoice': 'nunique',                                   # Frequency
    'TotalAmount': 'sum'                                    # Monetary
})

# Rename columns to English standards
rfm.columns = ['Recency', 'Frequency', 'Monetary']

print("--- RFM Metrics Calculated ---")
print(rfm.head())

# Save the RFM table
rfm.to_csv('../data/rfm_table.csv')

--- RFM Metrics Calculated ---
             Recency  Frequency  Monetary
Customer ID                              
12346            165         11    372.86
12347              3          2   1323.32
12348             74          1    222.16
12349             43          3   2671.14
12351             11          1    300.93


In [2]:
# 1. Scoring Logic: Divide metrics into 5 quantiles
# Note: For Recency, lower is better (5), for others higher is better (5)
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# 2. Combine scores to create an RFM Segment
rfm['RFM_Score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

print("--- RFM Scoring Completed ---")
display(rfm.head())

# 3. Save the scored data
rfm.to_csv('../data/rfm_scored.csv')

--- RFM Scoring Completed ---


,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Score
Customer ID,,,,,,,
12346,165,11,372.86,2,5,2,252
12347,3,2,1323.32,5,2,4,524
12348,74,1,222.16,2,1,1,211
12349,43,3,2671.14,3,3,5,335
12351,11,1,300.93,5,1,2,512


In [3]:
# 1. Define segments based on R and F scores
# This is a standard mapping for RFM analysis
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose Them',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

# 2. Apply the mapping to R and F scores
rfm['Segment'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

print("--- Customer Segmentation Completed ---")
print(rfm[['Segment', 'Recency', 'Frequency', 'Monetary']].head())

# 3. Final CSV for Visualization (Asadbek for Week 4)
rfm.to_csv('../data/customer_segments.csv')

--- Customer Segmentation Completed ---
                         Segment  Recency  Frequency  Monetary
Customer ID                                                   
12346           Can't Loose Them      165         11    372.86
12347        Potential Loyalists        3          2   1323.32
12348                Hibernating       74          1    222.16
12349             Need Attention       43          3   2671.14
12351              New Customers       11          1    300.93


In [4]:
import pandas as pd

# 1. Load the RFM table calculated previously
rfm = pd.read_csv('../data/rfm_table.csv', index_col='Customer ID')

# 2. RFM Scoring (1 to 5)
# Recency: Lower is better (5), so we use labels [5, 4, 3, 2, 1]
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Frequency and Monetary: Higher is better (5)
# We use rank(method='first') for Frequency to handle duplicate values in quantiles
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# 3. Define the Segments (Professional Industry Standard Mapping)
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose Them',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

# Apply segmentation based on R and F scores
rfm['Segment'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

print("--- RFM Segmentation Completed ---")
print(rfm[['Segment', 'R_score', 'F_score', 'M_score']].head())

# 4. Save for Asadbek (Visualization Specialist)
rfm.to_csv('../data/customer_segments.csv')
print("\n✅ Successfully saved: ../data/customer_segments.csv")

--- RFM Segmentation Completed ---
                         Segment R_score F_score M_score
Customer ID                                             
12346           Can't Loose Them       2       5       2
12347        Potential Loyalists       5       2       4
12348                Hibernating       2       1       1
12349             Need Attention       3       3       5
12351              New Customers       5       1       2

✅ Successfully saved: ../data/customer_segments.csv
